In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from scipy import stats

### Documentacion de stats model: https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLSResults.t_test.html#statsmodels.regression.linear_model.OLSResults.t_test

In [2]:
df_casa = pd.read_excel('./Ejemplo_Casa.xls')
display(df_casa.head())

,PRECIO,LOTE,CUARTOS,BANOS,PISOS,ENTRADA,REC,SOTANO,CALEF,AIRE,GARAGE,NBHD
0,42000,5850,3,1,2,1,0,1,0,0,1,0
1,38500,4000,2,1,1,1,0,0,0,0,0,0
2,49500,3060,3,1,1,1,0,0,0,0,0,0
3,60500,6650,3,1,2,1,1,0,0,0,0,0
4,61000,6360,2,1,1,1,0,0,0,0,0,0


In [3]:
y=df_casa['PRECIO']
X=sm.add_constant(df_casa.drop(columns='PRECIO'))

reg_casa=sm.OLS(y,X).fit()
display(reg_casa.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 PRECIO   R-squared:                       0.673
Model:                            OLS   Adj. R-squared:                  0.666
Method:                 Least Squares   F-statistic:                     99.97
Date:                Fri, 25 Jul 2025   Prob (F-statistic):          6.18e-122
Time:                        14:39:18   Log-Likelihood:                -6034.1
No. Observations:                 546   AIC:                         1.209e+04
Df Residuals:                     534   BIC:                         1.214e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -4038.3504   3409.471     -1.184      0.237   -1.07e+04    2659.271
LOTE           3.5463      0.350     10.124      0.000       2.858       4.234
CUARTOS     1832.0035   1047.000      1.750      0.081    -224.741    3888.748
BANOS       1.434e+04   1489.921      9.622      0.000    1.14e+04    1.73e+04
PISOS       6556.9457    925.290      7.086      0.000    4739.291    8374.600
ENTRADA     6687.7789   2045.246      3.270      0.001    2670.065    1.07e+04
REC         4511.2838   1899.958      2.374      0.018     778.976    8243.592
SOTANO      5452.3855   1588.024      3.433      0.001    2332.845    8571.926
CALEF       1.283e+04   3217.597      3.988      0.000    6510.706    1.92e+04
AIRE        1.263e+04   1555.021      8.124      0.000    9578.182    1.57e+04
GARAGE      4244.8290    840.544      5.050      0.000    2593.650    5896.008
NBHD        9369.5132   1669.091      5.614      0.000    6090.724    1.26e+04
==============================================================================
Omnibus:                       93.454   Durbin-Watson:                   1.604
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              247.620
Skew:                           0.853   Prob(JB):                     1.70e-54
Kurtosis:                       5.824   Cond. No.                     3.07e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.07e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Test de hipótesis
Tomo la regresora $\hat\beta$
$$
 H_0: \hat\beta=0
 \\
 H_1: \hat\beta <>0
$$
Donde reg=0 implica que no es significativo y <> que que si lo es, por lo tanto explica a X

In [4]:
#Hacemos un t-test para los cuartos

display(reg_casa.t_test("CUARTOS=0"))

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0          1832.0035   1047.000      1.750      0.081    -224.741    3888.748

Podemos reconstruir el IC de la siguiente forma

$$
\hat{\beta} \pm t_{\alpha/2, \, gl} \cdot \text{SE}(\hat{\beta})
$$

para este estimador la cuenta seria

$$
    1832 \pm 1.96 *1047
$$

El valor t que aprece en la regresión es el t observado, el t que usamos para el intervalo de confianza es el valor crítico para la distribución t-student, que como es muy grande (534) se asemeja al valor de la normal.

En función del p-value podemos ver que es significativo al 10% pero no al 5% porque es 0.081 (8%)

### Intervalo de confianza

In [22]:
display(reg_casa.conf_int(alpha=0.05).loc["LOTE"])

0    2.858168
1    4.234438
Name: LOTE, dtype: float64

### Test F
Estos test sirven para detectar multicolinealidad. Si el p-value es significativo==> rechazo H0. 
En este caso la H0 es que los coeficiente en su conjunto son 0, es decir existe multicolinealidad. 
En caso de NRH0 entonces HAY MULTICOLINEADAD si el p-value no es significativo.

In [14]:
print(reg_casa.f_test("CALEF=0, AIRE=0"))

<F test: F=36.888750867416185, p=9.856033740435273e-16, df_denom=534, df_num=2>


In [20]:
df_casa_filtrado=df_casa[df_casa['BANOS']!=4].copy()
y1=df_casa_filtrado['PRECIO']
X1=sm.add_constant(df_casa_filtrado.drop(columns='PRECIO'))


reg_casa_2=sm.OLS(y1,X1).fit()
print(reg_casa_2.summary())

                            OLS Regression Results                            
Dep. Variable:                 PRECIO   R-squared:                       0.665
Model:                            OLS   Adj. R-squared:                  0.658
Method:                 Least Squares   F-statistic:                     96.14
Date:                Sat, 26 Jul 2025   Prob (F-statistic):          7.67e-119
Time:                        06:14:07   Log-Likelihood:                -6022.2
No. Observations:                 545   AIC:                         1.207e+04
Df Residuals:                     533   BIC:                         1.212e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3509.7393   3419.390     -1.026      0.3

## Variables Dummy

In [ ]:
df_casa_filtrado['DB2']=(df_casa_filtrado['BANOS']==2).astype(int)
df_casa_filtrado['DB3']=(df_casa_filtrado['BANOS']==3).astype(int)

X2=sm.add_constant(df_casa_filtrado.drop(columns=['PRECIO','BANOS'])) # Elimino la columna BANOS porque ya tengo las dummies

reg_casa_3=sm.OLS(y1,X2).fit()
print(reg_casa_3.summary())


                            OLS Regression Results                            
Dep. Variable:                 PRECIO   R-squared:                       0.665
Model:                            OLS   Adj. R-squared:                  0.657
Method:                 Least Squares   F-statistic:                     88.01
Date:                Sat, 26 Jul 2025   Prob (F-statistic):          7.04e-118
Time:                        07:44:16   Log-Likelihood:                -6022.1
No. Observations:                 545   AIC:                         1.207e+04
Df Residuals:                     532   BIC:                         1.213e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.031e+04   3464.573      2.976      0.0

### Esto es para ver cuanto se valora un baño mas

In [32]:
print(reg_casa_3.f_test("DB3= 2*DB2"))

<F test: F=0.1815129172859376, p=0.6702485908495301, df_denom=532, df_num=1>


### Formula

In [31]:
reg_casa_4=smf.ols("PRECIO~LOTE+CUARTOS+C(BANOS)+PISOS+ENTRADA+REC+SOTANO+CALEF+AIRE+GARAGE+NBHD",
                   data=df_casa_filtrado).fit()

print(reg_casa_4.summary())

                            OLS Regression Results                            
Dep. Variable:                 PRECIO   R-squared:                       0.665
Model:                            OLS   Adj. R-squared:                  0.657
Method:                 Least Squares   F-statistic:                     88.01
Date:                Sat, 26 Jul 2025   Prob (F-statistic):          7.04e-118
Time:                        07:49:50   Log-Likelihood:                -6022.1
No. Observations:                 545   AIC:                         1.207e+04
Df Residuals:                     532   BIC:                         1.213e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      1.031e+04   3464.573      2.976

In [35]:
print(reg_casa_4.f_test('C(BANOS)[T.2]=0'))

<F test: F=60.98075593837472, p=3.087247239773227e-14, df_denom=532, df_num=1>


In [ ]:
## Interacciones entre variables
## La formula incorpora por defecto la constante


reg_casa_5=smf.ols('PRECIO ~ LOTE+CUARTOS+LOTE:NBHD',data=df_casa)



In [41]:
#Incluye lote, nbhd y lote*nbhd
reg_casa_6=smf.ols('PRECIO ~CUARTOS+LOTE*NBHD',data=df_casa).fit()

display(reg_casa_6.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 PRECIO   R-squared:                       0.409
Model:                            OLS   Adj. R-squared:                  0.405
Method:                 Least Squares   F-statistic:                     93.70
Date:                Sat, 26 Jul 2025   Prob (F-statistic):           1.62e-60
Time:                        10:35:35   Log-Likelihood:                -6195.6
No. Observations:                 546   AIC:                         1.240e+04
Df Residuals:                     541   BIC:                         1.242e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   7294.7262   4163.014      1.752      0.080    -882.925    1.55e+04
CUARTOS     1.027e+04   1212.646      8.470      0.000    7888.478    1.27e+04
LOTE           5.3083      0.504     10.539      0.000       4.319       6.298
NBHD        9343.4224   5666.371      1.649      0.100   -1787.362    2.05e+04
LOTE:NBHD      0.5926      0.917      0.646      0.519      -1.210       2.395
==============================================================================
Omnibus:                       84.951   Durbin-Watson:                   1.188
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              151.614
Skew:                           0.925   Prob(JB):                     1.20e-33
Kurtosis:                       4.801   Cond. No.                     3.94e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.94e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:

print(reg_casa_6.f_test('NBHD=0,LOTE:NBHD=0'))

<F test: F=17.85790812392496, p=3.087994730282173e-08, df_denom=541, df_num=2>
